## Import Libraries You Need

In [68]:
import requests
import pandas as pd
import numpy as np
import datetime
import pymysql
from ggplot import *
from configparser import ConfigParser
import json
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

In [43]:
#### OPTIONAL: IMPORT boto3 to download a useful file from S3 buacket
import boto3

## Grab Credentials Securely

### Using ConfigParser

In [69]:
config = ConfigParser()
config_file = 'config.ini'
config.read(config_file)

user = config['rds']['user']
password = config['rds']['pw']
hostname = config['rds']['host']
db_name = config['rds']['dbname']

api_key = config['quandl']['api_key']

## Connect to MySQL Database

In [92]:
connection = pymysql.connect(host=hostname,
                             user=user,
                             password=password,
                             db=db_name)

In [93]:
cursor = connection.cursor()

In [94]:
engine = create_engine('mysql://'+user+':'+password+'@'+hostname+':3306/'+db_name)

## Check out the documentation for the Quandl Zillow House Price API

Visit https://www.quandl.com/data/ZILLOW-Zillow-Real-Estate-Research/documentation/data-organization and review the documentation.

Now it's time to start querying! 

### 1) import the 'indicators.csv' file from the repo
I've already downloaded the file for you.


In [12]:
zillow_indicators = ### YOUR CODE HERE

##### Expected Output

In [14]:
zillow_indicators[:5]

,INDICATOR|CODE
0,Age of Inventory (Public)|AOIP
1,Buyer Seller Index|BSI
2,Days on Zillow (Public)|DOZP
3,Home Sales (NSA)|SALES
4,Home Sales (SA)|SASALES


### 2) split the 'INDICATOR|CODE'  column into two separate columns, INDICATOR and CODE
Note: There are many ways to accomplish this goal, and I encourage you to use your own research powers to find the one that makes the most sense to you. 

In [16]:
zillow_indicators['INDICATOR'], zillow_indicators['CODE'] = ### YOUR CODE HERE

##### Expected Output

In [17]:
zillow_indicators[:5]

,INDICATOR|CODE,INDICATOR,CODE
0,Age of Inventory (Public)|AOIP,Age of Inventory (Public),AOIP
1,Buyer Seller Index|BSI,Buyer Seller Index,BSI
2,Days on Zillow (Public)|DOZP,Days on Zillow (Public),DOZP
3,Home Sales (NSA)|SALES,Home Sales (NSA),SALES
4,Home Sales (SA)|SASALES,Home Sales (SA),SASALES


### 3) Get rid of the un-needed INDICATOR|CODE column

In [18]:
zillow_indicators  = ### YOUR CODE HERE

##### Expected Output

In [153]:
zillow_indicators[:5]

,INDICATOR,CODE
0,Age of Inventory (Public),AOIP
1,Buyer Seller Index,BSI
2,Days on Zillow (Public),DOZP
3,Home Sales (NSA),SALES
4,Home Sales (SA),SASALES


### 4) Query the Quandl API for the 'Median Listing Price - All Homes' in the city of Iowa City Since 1/1/2010
Consult the documentation on this page for the appropriate geographic codes:
https://www.quandl.com/data/ZILLOW-Zillow-Real-Estate-Research/documentation/data-organization

Note that the URL is constructed as: ZILLOW/{AREA_CATEGORY}{AREA_CODE}__{INDICATOR_CODE} (with single underscore)



In [25]:
base_url = 'https://www.quandl.com/api/v3/datasets/ZILLOW/'

In [57]:
url = base_url + ###YOUR CODE HERE + '&api_key=' +api_key

In [58]:
response = requests.get(url)

##### Expected Output

In [60]:
output_json = response.json()
output_json

{'dataset': {'collapse': None,
  'column_index': None,
  'column_names': ['Date', 'Value'],
  'data': [['2018-07-31', 267900.0],
   ['2018-06-30', 268500.0],
   ['2018-05-31', 274900.0],
   ['2018-04-30', 285000.0],
   ['2018-03-31', 285000.0],
   ['2018-02-28', 289900.0],
   ['2018-01-31', 290000.0],
   ['2017-12-31', 279900.0],
   ['2017-11-30', 279000.0],
   ['2017-10-31', 279000.0],
   ['2017-09-30', 289900.0],
   ['2017-08-31', 274900.0],
   ['2017-07-31', 275000.0],
   ['2017-06-30', 269950.0],
   ['2017-05-31', 265000.0],
   ['2017-04-30', 258950.0],
   ['2017-03-31', 264900.0],
   ['2017-02-28', 269900.0],
   ['2017-01-31', 264950.0],
   ['2016-12-31', 249900.0],
   ['2016-11-30', 248450.0],
   ['2016-10-31', 249950.0],
   ['2016-09-30', 250500.0],
   ['2016-08-31', 249900.0],
   ['2016-07-31', 237500.0],
   ['2016-06-30', 230000.0],
   ['2016-05-31', 231000.0],
   ['2016-04-30', 229900.0],
   ['2016-03-31', 225000.0],
   ['2016-02-29', 240000.0],
   ['2016-01-31', 249900.0],
 

### 5) Convert JSON Output to a DataFrame

In [64]:
data = ### YOUR CODE HERE
headers = ### YOUR CODE HERE
iowa_city_house_prices = pd.DataFrame(data=data,columns=headers)

##### Expected Output

In [65]:
iowa_city_house_prices[:10]

,Date,Value
0,2018-07-31,267900.0
1,2018-06-30,268500.0
2,2018-05-31,274900.0
3,2018-04-30,285000.0
4,2018-03-31,285000.0
5,2018-02-28,289900.0
6,2018-01-31,290000.0
7,2017-12-31,279900.0
8,2017-11-30,279000.0
9,2017-10-31,279000.0


### 6) Import All Cities file from zillow_cities.txt 
Note that this file is pipe-delimited ('|'). You might benefit from this documentation:
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

In [50]:
zillow_cities = ### YOUR CODE HERE

##### Expected Output

In [51]:
zillow_cities[:10]

,AREA,CODE
0,"Aaronsburg, PA",22759
1,"Abbeville, AL",4309
2,"Abbeville, GA",17572
3,"Abbeville, LA",14615
4,"Abbeville, MS",16597
5,"Abbeville, SC",24719
6,"Abbot, ME",19942
7,"Abbotsford, WI",6609
8,"Abbott, PA",20990
9,"Abbottstown, PA",21077


## 7) Subset zillow_cities for only Iowa Cities
You might make use of the str.contains method for columns in Pandas

In [114]:
iowa_cities = ### YOUR CODE HERE

##### Expected Output 

In [116]:
len(iowa_cities)

934

## 8) Loop through Iowa Cities, and Create a SQL Table with all Iowa City House Price Medians Since 1/1/2010
Make sure to handle missing cities from the API using the try... except... method!

In [155]:
for index, row in iowa_cities.iterrows():
    try:
        url = ### YOUR CODE HERE
        response = ### YOUR CODE HERE
        response_json = ### YOUR CODE HERE
        data = ### YOUR CODE HERE
        headers = ### YOUR CODE HERE
        city_house_prices = pd.DataFrame(data=data,columns=headers)
        city_house_prices['City'] = row['AREA']
        city_house_prices.to_sql(name='iowa_house_prices',con=engine,if_exists='append',index=False)
        print(row['AREA'],' extracted and uploaded')
    except Exception as ex:
        ### YOUR EXCEPTION HANDLING HERE

'dataset'
'dataset'
'dataset'
Adel, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
Albia, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
'dataset'
Algona, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
Altoona, IA  extracted and uploaded
'dataset'
'dataset'
Ames, IA  extracted and uploaded
Anamosa, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
Ankeny, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
Arnolds Park, IA  extracted and uploaded
'dataset'
Asbury, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
Atlantic, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'data

'dataset'
'dataset'
'dataset'
'dataset'
Ottumwa, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
Panora, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
Pella, IA  extracted and uploaded
'dataset'
'dataset'
Perry, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
Pleasant Hill, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
'dataset'
Polk City, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
Red Oak, IA  extracted and uploaded
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'dataset'
'data

##### Expected Output

In [156]:
iowa_city_data = pd.read_sql_query("SELECT * FROM iowa_house_prices LIMIT 100",connection)

In [157]:
iowa_city_data[:10]

,Date,Value,City
0,2018-08-31,307150.0,"Adel, IA"
1,2018-07-31,291400.0,"Adel, IA"
2,2018-06-30,291400.0,"Adel, IA"
3,2018-05-31,287655.0,"Adel, IA"
4,2018-04-30,286000.0,"Adel, IA"
5,2018-03-31,284700.0,"Adel, IA"
6,2018-02-28,283615.0,"Adel, IA"
7,2018-01-31,274900.0,"Adel, IA"
8,2017-12-31,287000.0,"Adel, IA"
9,2017-11-30,287000.0,"Adel, IA"


## Now that you have your data...
## 9) Which city had the highest month-over-month growth in median house prices? In which month?
Take a look at Pandas DataFrame.groupby() and Series.pct_change()

In [158]:
iowa_prices = pd.read_sql_query("""/*YOUR SQL QUERY HERE*/
                                --SELECT ... 
                                """
                                ,connection)

In [160]:
iowa_prices['pct_change'] = ### YOUR CODE HERE
iowa_prices[### YOUR CODE HERE]

## 10) Create a visualization of the 10 fastest growing Iowa cities for house prices

In [1]:
iowa_prices.sort_values(by=['pct_change'],ascending=False,inplace=True)

In [163]:
top_10_cities = ### YOUR CODE HERE

In [164]:
top_ten_prices = ### YOUR CODE HERE

In [150]:
top_ten_prices = ### FILL NA pct_change VALUES WITH 0
top_ten_prices['Date'] = ### CONVERT DATE TO DATETIME

In [1]:
from ggplot import *
#ggplot(### YOUR CODE HERE) + geom_line()